In [ ]:
%load_ext dotenv
%dotenv

import pandas as pd
import os
import json
import hashlib
import pgOperation
from sqlalchemy.types import VARCHAR, Float, Integer, Date, Numeric
pd.options.mode.chained_assignment = None

# All configuration from environment variables
pg = pgOperation.PgOperation(
    ip=os.getenv('PG_HOST'),
    port=os.getenv('PG_PORT'),
    user=os.getenv('PG_USER'),
    pwd=os.getenv('PG_PASSWORD'),
    db=os.getenv('PG_DATABASE'),
    schema=os.getenv('PG_SCHEMA', 'public')
)

In [ ]:
# List all tables and export snapshots to CSV
tables = pg.listTables()
print(f"Tables in schema '{pg.schema}':")
for t in tables:
    print(f"  - {t}")

# Export all tables to snapshot folder (first 10 rows each)
snapshots = pg.exportAllTablesToSnapshot('snapshot', limit=10)

In [ ]:
# Demo: Read first 5 rows from game_results table
demo_df = pg.readSql('SELECT * FROM game_results LIMIT 5')
print(f"Successfully connected to database: {os.getenv('PG_DATABASE')}")
print(f"Table: game_results (first 5 rows)")
demo_df

In [ ]:
game_result_df = pg.readTable('game_results')
game_result_df['createtime'] = pd.to_datetime(game_result_df['createtime'])
pg.writeDfToPg(game_result_df, 'ods_main_games') # 之前的ods_main_games

In [ ]:
def split_corporation_column(df):
    # 创建输出的DataFrame的副本，以便不直接修改原始的DataFrame
    new_df = df.copy()

    # 为了不影响原有的corporation2值，我们先创建一个掩码来找出corporation2为空的行
    mask = new_df['corporation2'].isnull()

    # 对corporation列进行拆分，但只拆分那些corporation2为空的行
    split_df = new_df.loc[mask, 'corporation'].str.split('\|', expand=True)

    # 更新corporation列为拆分后的第一个元素，即'|'左边的数据
    # 并且创建新的corporation1列来存储原始的corporation列的值
    # new_df.loc[mask, 'corporation1'] = split_df[0]
    new_df.loc[mask, 'corporation'] = split_df[0]

    # 将拆分后的第二个和第三个元素分别赋值给corporation2和corporation3列
    new_df.loc[mask, 'corporation2'] = split_df[1]
    new_df.loc[mask, 'corporation3'] = split_df[2]
    new_df.loc[mask, 'corporation4'] = split_df[3]

    # 将NaN替换为你想要的空值表示，例如空字符串''
    new_df.fillna('', inplace=True)

    return new_df

In [ ]:
game_result_df['scores'] = game_result_df['scores'].apply(lambda x:eval(x))
pd.json_normalize(game_result_df['scores'])

tmp_game_df = pd.melt(
    # pd.concat([pd.DataFrame(pd.json_normalize(x)) for x in game_df['scores']],ignore_index=True)
    pd.json_normalize(game_result_df['scores'])
    .reset_index(), id_vars='index', value_name='player_data').dropna(subset=['player_data']).set_index('index', drop=True)
flat_game_df = (pd.json_normalize(tmp_game_df['player_data']).join(tmp_game_df.reset_index(drop=False)['index']).set_index('index')) \
    .join(game_result_df.drop(['seed_game_id', 'game_options', 'scores'], axis=1)).reset_index()
flat_game_df['position'] = flat_game_df.sort_index() \
            .groupby(['game_id']) \
            .cumcount() + 1
flat_game_df['rank'] = flat_game_df.sort_values(['playerScore'], ascending=[False]) \
            .groupby(['game_id']) \
            .cumcount() + 1
# remove firstletter in flat_game_df['player'] if start with ~ or @
flat_game_df['player'] = flat_game_df['player'].apply(lambda x: x[1:] if x[0] in ['~', '@', '～'] else x)
# flat_game_df player to be lowercase
flat_game_df['player'] = flat_game_df['player'].apply(lambda x: x.lower())
flat_game_df.drop('index', axis=1, inplace=True)

flat_game_df = split_corporation_column(flat_game_df)



In [ ]:
pg.writeDfToPg(flat_game_df, 'ods_game_results')

sql = """
    select *
    from ods_game_results
    where createtime between '2022-10-15' and '2024-01-01'
    and player not in ('Green', '1', '2', '3', 'Red', 'Blue', '绿色', '红色', '黄色', '蓝色', 'Yellow', '4')
    and generations <= 13;
"""
pg.writeDfToPg(pg.readSql(sql), 'ods_game_2023_results')

# TODO: 按照玩家数量和次数做一个聚合
pg.readSql("select * from ods_game_2023_results where corporation4 <> ''").to_csv('./打出了4公司的玩家.csv', index=False) # 有4个公司的玩家
pg.readSql("select * from ods_game_2023_results where corporation3 <> '' and corporation4 <= ''").to_csv('./打出了3公司的玩家.csv', index=False) # 有3个公司的玩家


In [ ]:
# game_tables = ["games_2022.08_2022.10", "games_2022.11_2023.02", "games_2023.03_2023.06"]

sql = """
select distinct *
from (
    select *
    from "games_2022.08_2022.10"
    where status = 'finished'
    and save_id > 0
    and createtime > '2022-10-15'
    union
    select *
    from "games_2022.11_2023.02"
    where status = 'finished'
    and save_id > 0
    union
    select *
    from "games_2023.03_2023.06"
    where status = 'finished'
    and save_id > 0
    union
    select *
    from games_2
    where status = 'finished'
    and save_id > 0
) as unique_games;
"""
game_df = pg.readSql(sql) \
            .drop_duplicates(subset=['game_id'], keep='last', inplace=False)
game_df['createtime'] = pd.to_datetime(game_df['createtime'])

pg.writeDfToPg(game_df, 'ods_detail_games')

In [ ]:
game_df

## 后续计算

In [ ]:
detail_df = game_df

# 选择所有已完成数据
false = False
true = True
null = None
def getCardData(detail_df, game_df):
    df1a = detail_df

    #  关联快照表，取所需字段，即game_df可以通过streamlit选项动态调整
    df2 = game_df.merge(df1a,how='left',on=['game_id', 'game_id'], suffixes=['','_drop'], indicator=True).query('_merge == "both"')

    # df2 = df1b[df1b['players'] == player_number]
    # 修改需要展开字段的类型为list
    df2['game'] = df2['game'].apply(lambda x:eval(x))
    mid = pd.json_normalize(df2['game'],record_path=['players', 'playedCards'], meta=['id',  ['players','terraformRating'], ['players','name']], sep='_') \
        .fillna('')
        # .drop('targetCards', axis=1)

    # mid['players_name'] = mid['players_name'].apply(lambda x: x.lower())
    mid = mid.astype(str)

    # mid['players'] = player_number
    return mid

In [ ]:

mid = getCardData(detail_df, pg.readTable('ods_main_games'))
pg.writeDfToPg(mid, 'ods_card_results')
# mid.to_sql('ods_card_results', con=ana, if_exists='replace')
# mid2 = getCardData(detail_df, game_df, 2)
# mid = pd.concat([mid2, mid4])



In [ ]:
mid

In [ ]:
pg.run("drop table if exists dwd_cards;")
sql2 = """
    create table dwd_cards as
    select game_id,
        player,
        corporation,
        corporation2,
        corporation3,
        corporation4,
        name                    as card_name,
        "playerScore",
        cast("players_terraformRating" as INTEGER) as tr,
        players,
        generations,
        position,
        rank,
        createtime,
        cast("resourceCount" as INTEGER) as resourceCount,
        "cloneTag",
        -- isDisabled,
        "bonusResource",
        "userId"
    from ods_card_results
            inner join ods_game_2023_results on ods_card_results.id = ods_game_2023_results.game_id and
                                        lower(ods_card_results.players_name) = lower(ods_game_2023_results.player)
"""
pg.run(sql2)
# pg.writeDfToPg(mid, 'ods_card_results')

In [ ]:
# 前序表
pg.run("drop table if exists dim_prelude;")
sql = """
    create table dim_prelude as
    select card_name, count(*)
    from (
            select *, row_number() over (partition by game_id, player) as rn
            from dwd_cards) as a
    where rn = 0
    or rn = 1
    group by card_name
    having count(*) > 2
"""
pg.run(sql)

In [ ]:
# tr表
pg.run("drop table if exists dwd_tr;")
sql = """
    create table dwd_tr as
    select a.*, tr + a.generations - 21 as increase_tr
    from (select *, row_number() over (partition by game_id, player) as rn from dwd_cards) as a
    where rn = 1;
"""
pg.run(sql)

In [ ]:
# 玩家打牌表，含卡牌类别
pg.run("drop table if exists dwd_user_cards;")
sql = """
create table dwd_user_cards as
select a.*, b.count, case when b.card_name is not null then 'prelude' else 'project' end as card_type
from (select user_name, card_name, count(*) as total
      from (select a.*, coalesce(c.name, a.player) as user_name
            from dwd_cards
                     as a
                     left join ods_user_alias as c
                               on a.player = c.name
            where createtime between '2022-10-15' and '2024-12-30'
              and a.player not in
                  ('green', '1', '2', '3', 'red', 'blue', '绿色', '红色', '黄色', '蓝色', 'yellow', '4')) as a
      group by card_name, user_name
      having count(*) >= 5
      order by total desc) as a
         left join dim_prelude as b
                   on a.card_name = b.card_name;
"""
pg.run(sql)

In [ ]:
sql = """
    select corporation,
        players,
        avg(rank)                           as rank,
        avg(playerScore)                    as score,
        avg(generations)                    as generations,
        count(corporation)                  as total_games
    from (select corporation,
                playerScore,
                players,
                rank,
                generations
        from ods_game_2023_results
        union all
        select corporation2,
                playerScore,
                players,
                rank,
                generations
        from ods_game_2023_results
        where corporation2 <> '')
    group by corporation, players
    order by players desc, rank desc;
"""

sql2 = """
select card_name,
       players,
               avg(rank)                           as rank,
        avg(playerScore)                    as score,
        avg(generations)                    as generations,
        count(corporation)                  as total_games
from dwd_cards
group by card_name, players
having players in (4)
-- and (card_name like '%Border Che%' or card_name like '%Fencing%' or card_name like '%Antigravity Experiment%' or card_name like '%Hay Maker%' or card_name like '%Venus Uni%'
--     or card_name like '%Fleet Re%' or card_name like '%Smuggling%')
order by players, rank
;
"""

sql3 = """
select player,
       players,
               avg(rank)                           as rank,
        avg(playerScore)                    as score,
        avg(generations)                    as generations,
        count(corporation)                  as total_games
from ods_game_2023_results
group by player, players
having players in (4)
-- and (card_name like '%Border Che%' or card_name like '%Fencing%' or card_name like '%Antigravity Experiment%' or card_name like '%Hay Maker%' or card_name like '%Venus Uni%'
--     or card_name like '%Fleet Re%' or card_name like '%Smuggling%')
order by players, total_games desc
;
"""
mid1 = mid.loc[:,['id','players_name','name','resourceCount','players_terraformRating', 'players']]
mid1['rn'] = mid1 \
            .groupby(['id','players_name', 'players']) \
            .cumcount() + 1
# mid
# TODO  需要关联处理后的game_result表，获取position, gens, createtime等数据, 只能通过merge实现



In [ ]:
mid

In [ ]:
def getPlayerNumResult(df, player_num = 4):
    """
    主键: game_id, player
    """
    df = df.loc[df['players'] == player_num].reset_index(drop=True)
    for i in range(1, player_num+1):
        player_idx = 'player'+str(i)
        # player_df_pre = df[player_idx].apply(lambda x:eval(x))
        # print(player_idx)
        # player_df = pd.json_normalize(player_df_pre).reset_index(drop=True)
        player_df = pd.json_normalize(df[player_idx]).reset_index(drop=True)
        if i == 1:
            res = pd.concat([df,player_df.reindex(df.index)],axis=1)
            print((res.loc[pd.isna(res['player']) == False]).shape[0])
        else:
            mid = pd.concat([df,player_df.reindex(df.index)],axis=1)
            res = pd.concat([res, mid],axis=0, ignore_index=True)
            # print((mid.loc[pd.isna(mid['player']) == False]).shape[0])
        # df = pd.concat([df, pd.json_normalize(df[player_idx])],axis=1)
    res.drop(['player'+str(i) for i in range(1, 7)], axis=1, inplace=True)
    res['count'] = 1
    res['players'] = player_num
    return res

def getCardRank(df):
    """
    对card做聚合, 取打出胜率以及打出次数
    """
    # res = df.query('rn not in (1,2)') \
    res = df \
    .groupby(['name', 'players']) \
    .agg(
        position = ('position', 'mean'),
        playerScore = ('playerScore', 'mean'),
        generations = ('generations', 'mean'),
        total = ('count', 'sum')
    ) \
    .dropna().sort_values('position').reset_index()
    cn = pd.read_csv('./中文翻译.csv')
    res = res.merge(cn, left_on = ['name'], right_on = ['en'], how = 'left')
    first_column = res.pop('cn')

    res.insert(0, 'cn', first_column)
    res.loc[pd.isna(res['cn'])==True,'cn'] = res.loc[pd.isna(res['cn'])==True,'name']
    res = res.drop('en', axis = 1)
    return res

def getPlayersCardRank(df):
    """
    对card做聚合, 取打出胜率以及打出次数
    """
    res = df.query('rn not in (1,2)') \
    .groupby(['player', 'name', 'players']) \
    .agg(
        position = ('position', 'mean'),
        playerScore = ('playerScore', 'mean'),
        generations = ('generations', 'mean'),
        total = ('count', 'sum'),
        sum_position = ('position', 'sum'),
        sum_playerScore = ('playerScore', 'sum'),
        sum_generations = ('generations', 'sum')
    ) \
    .dropna().sort_values('position').reset_index()
    cn = pd.read_csv('./中文翻译.csv')
    res = res.merge(cn, left_on = ['name'], right_on = ['en'], how = 'left')
    first_column = res.pop('cn')

    res.insert(0, 'cn', first_column)
    res.loc[pd.isna(res['cn'])==True,'cn'] = res.loc[pd.isna(res['cn'])==True,'name']
    res = res.drop('en', axis = 1)
    return res

player_df4 = getPlayerNumResult(df4, 4) # 主键: game_id, player
player_df2 = getPlayerNumResult(df4, 2) # 主键: game_id, player
player_df = pd.concat([player_df4, player_df2])

card_df = mid1.merge(player_df, how='left', left_on=['id', 'players_name'], right_on=['game_id', 'player'], suffixes=['','_drop'], indicator=True).query('_merge == "both"')

card_df_group = getCardRank(card_df)
card_df.to_csv('CardDetail.csv', index=False)
getPlayersCardRank(card_df).to_csv('playersCardRank.csv', index=False)
card_df_group.sort_values('total',ascending=False).to_csv('./allCardsRank.csv', index = False)